In [2]:
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers 
import mne_nirs
import nilearn
import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
from nilearn.datasets import fetch_development_fmri
from fsl.wrappers import fslmaths
import glob

In [3]:
%gui wx

Gtk-Message: 18:22:14.760: Failed to load module "canberra-gtk-module"


In [4]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

18:22:16: Debug: Adding duplicate image handler for 'Windows bitmap file'
18:22:16: Debug: Adding duplicate animation handler for '1' type
18:22:16: Debug: Adding duplicate animation handler for '2' type
18:22:17: Debug: Adding duplicate image handler for 'Windows bitmap file'
18:22:17: Debug: Adding duplicate animation handler for '1' type
18:22:17: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:441): Gtk-CRITICAL **: 18:22:17.630: gtk_window_resize: assertion 'height > 0' failed


In [8]:

# creating derivative root
deriv_root = op.join( notebook_dir, "dataset", "derivatives")
mkdir_no_exist(deriv_root)

# creating preprocessing root
preproc_root = op.join(deriv_root,'preprocessed_data')
mkdir_no_exist(preproc_root)
bids_root = op.join(notebook_dir, "dataset")

anat_path = op.join(bids_root, "T1w", "T1w.nii.gz")

In [5]:
fsleyesDisplay.load(anat_path)

In [ ]:
# os.system("bet")

In [9]:
# skull stripping
betted_path = op.join(preproc_root, "T1w", "T1w_skull.nii.gz")

In [ ]:
os.system('bet {} {} -m {}'.format(anat_path, betted_path, '-R' ))

In [9]:
fsleyesDisplay.load(betted_path)

In [30]:
# fsleyesDisplay.resetOverlays()
# fsleyesDisplay.load(op.join(preproc_root, "T1w", "T1w_skull_mask.nii.gz"))

In [10]:
os.system("fast")


Part of FSL (ID: "")
FAST 
Copyright(c) 2004-2012, University of Oxford

Usage: 
fast [options] file(s)

Optional arguments (You may optionally specify one or more of):
	-n,--class	number of tissue-type classes; default=3
	-I,--iter	number of main-loop iterations during bias-field removal; default=4
	-l,--lowpass	bias field smoothing extent (FWHM) in mm; default=20
	-t,--type	type of image 1=T1, 2=T2, 3=PD; default=T1
	-f,--fHard	initial segmentation spatial smoothness (during bias field estimation); default=0.02
	-g,--segments	outputs a separate binary image for each tissue type
	-a <standard2input.mat> initialise using priors; you must supply a FLIRT transform
	-A <prior1> <prior2> <prior3>    alternative prior images
	--nopve	turn off PVE (partial volume estimation)
	-b		output estimated bias field
	-B		output bias-corrected image
	-N,--nobias	do not remove bias field
	-S,--channels	number of input images (channels); default 1
	-o,--out	output basename
	-P,--Prior	use priors throug

256

In [17]:
# tissue segmentation

fast_path = op.join(preproc_root, "T1w", "T1w_fast")
# nome anomalo --> output di fast ha _skull_ e non _fast_
os.system('fast {} {}'.format(fast_path, betted_path))

0

In [18]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'T1w','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'T1w','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'T1w','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

In [10]:
# checking what does RL and LR mean

func_LR_path = op.join(bids_root, 'fMRI', 'tfMRI_MOTOR_LR','tfMRI_MOTOR_LR.nii')
func_RL_path = op.join(bids_root, 'fMRI', 'tfMRI_MOTOR_RL','tfMRI_MOTOR_RL.nii')

# fsleyesDisplay.resetOverlays()
# fsleyesDisplay.load(func_LR_path)
# fsleyesDisplay.load(func_RL_path)
# same reference

In [ ]:
# scaling variance to one
import numpy as np
temp = nib.load(func_LR_path)
LR_func_data= temp.get_fdata()
std_LR=LR_func_data.std(axis=(0,1,2))
LR_norm = LR_func_data / std_LR
LR_norm_img = nib.Nifti1Image(LR_norm.astype(np.uint8), temp.affine, temp.header)
# LR_path = op.join(preproc_root, 'fMRI','tfMRI_MOTOR_LR_norm.nii')
func_preproc_path = op.join(preproc_root, 'fMRI')
mkdir_no_exist(func_preproc_path)
nib.save(LR_norm_img, op.join(func_preproc_path,'tfMRI_MOTOR_LR_norm.nii'))

temp = nib.load(func_RL_path)
RL_func_data= temp.get_fdata()
std_RL=RL_func_data.std(axis=(0,1,2))
RL_norm = RL_func_data / std_RL
RL_norm_img = nib.Nifti1Image(RL_norm.astype(np.uint8), temp.affine, temp.header)
# RL_path = op.join(preproc_root, 'fMRI','tfMRI_MOTOR_RL_norm.nii')
nib.save(RL_norm_img, op.join(func_preproc_path,'tfMRI_MOTOR_RL_norm.nii'))

In [35]:
# concatenating runs
print(LR_norm.std())

1.000000021161999


In [22]:
subprocess.run(['fslmerge', '-t', merged_phase_imgs, 
                    op.join(fmap_path, 'sub-001_acq-task_dir-AP_epi.nii.gz'), 
                    op.join(fmap_path, 'sub-001_acq-task_dir-PA_epi.nii.gz')])

sh: 1: merge: not found


32512